#### Import relevant packages and functions 

In [2]:
import cv2
import numpy as np
import csv
#import codec
from PIL import Image
import pandas as pd
import time
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
import tensorflow as tf
from sklearn import preprocessing
import keras
from keras.models import Model
# from keras.models import Sequential
# from keras.layers import Flatten, Dense, Lambda
# from keras.layers import Convolution2D, MaxPooling2D, Cropping2D
# from keras.optimizers import Adamax, Adam, SGD
# from keras.layers.normalization import BatchNormalization
# from keras.layers.core import Dropout, Activation
# from keras.callbacks import ReduceLROnPlateau
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import LabelEncoder
# from keras.layers import GlobalAveragePooling2D, Conv2D
# from keras.initializers import TruncatedNormal
# from keras.regularizers import l1, l2
#from get_results import plot_roc_auc, plot_confusion_matrix
# from keras.callbacks import ModelCheckpoint 
# from keras.utils import to_categorical
from keras.models import load_model
import keras.backend as K

Using TensorFlow backend.


In [4]:
from keras.applications import MobileNet
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.applications.mobilenet import preprocess_input

#### Assign unique integer to each class

In [5]:
def classfinder(class_string):
    class_int = 0;
    if class_string == "FWD" :
        class_int = 0
    elif class_string == "R" :
        class_int = 1
    elif class_string == "TR" :
        class_int = 2   
    elif class_string == "T" :
        class_int = 3
    elif class_string == "TL" :
        class_int = 4
    elif class_string == "L" :
        class_int = 5
    elif class_string == "BL" :
        class_int = 6
    elif class_string == "B" :
        class_int = 7
    elif class_string == "BR" :
        class_int = 8 
      
    return class_int    

#### Load the saved Models

In [6]:
model_left = load_model('Models/MN_Aug-21_Left.h5')
model_right = load_model('Models/MN_Aug-21_Right.h5')

#### Load data from csv

In [7]:
vdf1 = pd.read_csv('filepaths/confusion_matrix_4_10_18.csv')

df_modeltest = vdf1

test_lines = []
print(type(test_lines))
test_lines = df_modeltest[df_modeltest.columns[0]].tolist()
df_modeltest["label"] = ""
df_modeltest["side"] = ""
df_modeltest["left_net_op"] = ""
df_modeltest["right_net_op"] = ""

for (i,line) in enumerate(test_lines):
    line = line.split("_")
    class_string = line[-2]
    side_string = line[-1]
    df_modeltest["label"][i] = class_string
    df_modeltest["side"][i] = side_string

df_modeltest.label.value_counts()

<class 'list'>


FWD    1214
TR      776
BR      681
BL      670
TL      633
L       290
R       286
B       255
T       241
Name: label, dtype: int64

In [8]:
equi_df = df_modeltest.groupby(['label']).apply(lambda x: x.sample(241, replace = True)).reset_index(drop = True)
print('New Data Size:', equi_df.shape[0], 'Old Size:', df_modeltest.shape[0])
equi_df.label.value_counts()

New Data Size: 2169 Old Size: 5046


TL     241
BL     241
BR     241
T      241
TR     241
R      241
L      241
FWD    241
B      241
Name: label, dtype: int64

In [9]:
# test_data = df_modeltest
test_data = equi_df
# check_pic = {"":["FB45/Left/350492_enh4_T_0.png","FB45/Right/350492_enh4_T_0.png"], 
#              'label': ["T", "T"], 
#              'side' : ["0.png", "1.png"]}
# test_data = pd.DataFrame(data=check_pic)
test_data.head()

,0,label,side,left_net_op,right_net_op
0,FB65\Left\74251_enh15_B_0.png,B,0.png,,
1,FB34\Left\369646_enh14_B_0.png,B,0.png,,
2,FB18\Left\104982_enh11_B_0.png,B,0.png,,
3,FB114\Left\177516_enh10_B_0.png,B,0.png,,
4,FB34\Left\369646_enh11_B_0.png,B,0.png,,


In [10]:
def findSecondBest(pred_list):
    pred_list = pred_list[0]
    preds = np.argsort(pred_list)[-2]
#     print(preds)
    return preds

def findThirdBest(pred_list):
    pred_list = pred_list[0]
    preds = np.argsort(pred_list)[-3]
    return preds
    

In [ ]:
no_of_lines = 1 # len(test_data)
count = 0
c1 = 0
c2 = 0
c3 = 0
c4 = 0
for index in range(0, no_of_lines):
    source_path = test_data.iloc[index][0]
    filename = source_path
    img = Image.open(source_path)
    img.load()
    img = img.resize((128, 128), Image.ANTIALIAS) 
    img_c = img.copy()
    image = np.asarray(img_c)
    image = preprocess_input(image)
    print(source_path)
    side = test_data.iloc[index][2]
    if side == '0.png':
        X = model_left.predict(np.array(image, ndmin = 4))
        XL = X
    if side == '1.png':
        X = model_right.predict(np.array(image, ndmin = 4))
        XR = X
    secBest = findSecondBest(X)
    thirdBest = findThirdBest(X)
    print(X)
    predicted_class = np.argmax(X)
    print("predicted classes: ", predicted_class, secBest, thirdBest)
    actual_class = classfinder((test_data.iloc[index][1]))
    print("actual class: ", actual_class)
#     if predicted_class == actual_class or (predicted_class + 1) % 9 == actual_class or (predicted_class - 1) % 9 == actual_class:
#         count = count + 1
#     if predicted_class == actual_class or actual_class == secBest or actual_class == thirdBest:
    if predicted_class == actual_class:
        count = count + 1
        c1 = c1 + 1
    elif (predicted_class == 0 and secBest == actual_class):
        print("candidate for c2")
        count = count + 1
        c2 = c2 + 1
#     elif actual_class != 0 and ( (predicted_class + 1) % 9 == actual_class or (predicted_class - 1) % 9 == actual_class):
    elif actual_class !=0:
        if actual_class == 1 and (predicted_class == 2 or predicted_class == 8):
            print("candidate for c3")
            count = count + 1
            c3 = c3 + 1
        elif actual_class == 8 and (predicted_class == 7 or predicted_class == 1):
            print("candidate for c3")
            count = count + 1
            c3 = c3 + 1
        elif (predicted_class + 1) % 9 == actual_class or (predicted_class - 1) % 9 == actual_class:
            print("candidate for c4")
            count = count + 1
            c4= c4+ 1

print(count, no_of_lines, count/no_of_lines)
print(c1,c2,c3,c4)
print()

XLR = XL + XR
print(XLR)
print(np.argmax(XLR), findSecondBest(XLR), findThirdBest(XLR))

In [11]:
model_right.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 129, 129, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128       
__________

In [12]:
layer_name_left = 'global_average_pooling2d_6'
intermediate_layer_model_left = Model(inputs=model_left.input,
                                 outputs=model_left.get_layer(layer_name_left).output)

layer_name_right = 'global_average_pooling2d_5'
intermediate_layer_model_right = Model(inputs=model_right.input,
                                 outputs=model_right.get_layer(layer_name_right).output)

In [13]:
no_of_lines = len(test_data)
no_of_test_cases = no_of_lines
count,c1,c2,c3,c4,second_count,fwd_count,percentage_count=0,0,0,0,0,0,0,0
predictions_list = list()
ground_truth_list = list()
for index in range(0, no_of_lines): 
    source_path = test_data.iloc[index][0]
    filename = "Eyes_dp\\" + source_path
    try: 
        img = Image.open(filename)
    except IOError:
        print("Error Left")
        no_of_test_cases = no_of_test_cases - 1
        continue
    img.load()
    img = img.resize((128, 128), Image.ANTIALIAS) 
    img_c = img.copy()
    image = np.asarray(img_c)
    image = preprocess_input(image)
#     print(source_path)
    
    intermediate_output_left = intermediate_layer_model_left.predict(np.array(image, ndmin = 4))
#     print(type(intermediate_output_left), type(intermediate_output_left[0]))
    test_data.iloc[index][3] = intermediate_output_left[0].tolist()
#     submission = pd.DataFrame(intermediate_output_left.tolist()[0]).to_csv('1000_classes_left.csv', index=False)
   
#     XL = model_left.predict(np.array(image, ndmin = 4))       
    newfilename = filename.split("\\")
    newfilename[-2] = "Right"
    name_split = newfilename[-1].split("_")
    name_split[-1] = "1.png"
    newfilename[-1] = "_".join(name_split)
    newfilename = "\\".join(newfilename)
    try: 
        img = Image.open(newfilename)
    except IOError:
        try:
            newfilename = filename.split("\\")
            newfilename[-2] = "Flipped_to_right"
            name_split = newfilename[-1].split("_")
            name_split[-1] = "01.png"
            newfilename[-1] = "_".join(name_split)
            newfilename = "\\".join(newfilename)
            img = Image.open(newfilename)
        except:
            print("Error Right")
#             test_data = test_data.drop(test_data.index[index], inplace=True)
            continue
#     print(newfilename)
    img.load()
    img = img.resize((128, 128), Image.ANTIALIAS) 
    img_c = img.copy()
    image = np.asarray(img_c)
    image = preprocess_input(image)
    
    intermediate_output_right = intermediate_layer_model_right.predict(np.array(image, ndmin = 4))
    test_data.iloc[index][4] = intermediate_output_right[0].tolist()
#     submission = pd.DataFrame(intermediate_output_right.tolist()[0]).to_csv('1000_classes_right.csv', index=False)
    
#     XR = model_right.predict(np.array(image, ndmin = 4))

# print(count, no_of_test_cases, count/no_of_test_cases)
# print(c1,c2,c3,c4)
# print()

# temp_df = test_data.iloc[0:5]
print(test_data.shape)
# sub = temp_df.to_pickle("test_df.pkl")

Error Right
Error Right
Error Right
Error Right
Error Right
Error Right
Error Right
Error Right
(2169, 5)


In [14]:
sub = test_data.to_pickle("test_df_equi_26_11.pkl")

In [12]:
from sklearn.utils import shuffle
shuffled_test_df = shuffle(test_data)

In [14]:
sub = shuffled_test_df.to_pickle("shuffled_test_df.pkl")

## With Imagenet weights

## OLD CODE

In [ ]:
print(second_count, fwd_count, percentage_count)

In [ ]:
dict_ = {"pred": predictions_list, 
             'truth': ground_truth_list}
df = pd.DataFrame(data=dict_)


In [ ]:
norm_df = df.groupby(['pred']).apply(lambda x: x.sample(250, replace = True)).reset_index(drop = True)
print('New Data Size:', norm_df.shape[0], 'Old Size:', df.shape[0])

In [ ]:
import sklearn.metrics as metrics


In [ ]:
confusion_matrix = metrics.confusion_matrix(y_true=norm_df[norm_df.columns[1]].tolist(), 
                                            y_pred=norm_df[norm_df.columns[0]].tolist())

# confusion_matrix = metrics.confusion_matrix(y_pred=predictions_list,
#                                            y_true=ground_truth_list)

In [ ]:
counts_list = df.pred.value_counts().sort_index().tolist()
print(type(counts_list))
print(counts_list)

In [ ]:
cm = confusion_matrix.copy()
for i in range(9):
    for j in range(9):
        cm[i][j] = float(confusion_matrix[i][j])  / counts_list[j] * 100
        
# cm[0][0] = 5.43

In [ ]:
print(cm)
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
plt.show()

In [ ]:
cnf_matrix = metrics.confusion_matrix(predictions_list, ground_truth_list)

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
plt.figure(figsize=(10,10))
class_list = list(range(9))
plot_confusion_matrix(cnf_matrix, classes=class_list, normalize=True)